# **Core Stock Data EDA for NVDA Ticker**
## In this notebook we will examine only the Nvidia stock for the periods we have selected for this project (03-14-2019 through 08-15-2024), and see what we can derive from it through our plots.  We will look at each of our core stock tickers separately in order to gain a better analysis.

#### Let's start by bringing in the libraries and logic necessary for reading in our file.

In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from scipy.stats import linregress
from scipy.stats import gaussian_kde

#### Now let's read in our data that we need for this notebook.

In [2]:
# Now let's access the main core_stock_data.csv file
csv_path = os.path.join(project_root, 'data', 'core_stock_data.csv')
core_stock_data = pd.read_csv(csv_path, parse_dates=['Date'], index_col= 'Date')
core_stock_data.head()

,Close_core,Volume_core,Open_core,High_core,Low_core,SMA_core,EMA_core,RSI_core,BBM_core,BBU_core,...,ATR_14_core,Stoch_K_core,Stoch_D_core,Momentum_1_core,Momentum_3_core,Momentum_7_core,Momentum_30_core,Momentum_50_core,OBV_core,Ticker
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-14,45.932499,94318000,45.974998,46.025002,45.639999,41.35925,42.219051,75.741602,41.35925,46.695085,...,0.700179,97.465683,90.860103,0.504997,1.207500,2.049999,4.619999,7.049999,1592190800,AAPL
2019-03-15,46.529999,156171600,46.212502,46.832500,45.935001,41.50025,42.388107,76.985910,41.50025,47.003365,...,0.712679,93.213648,93.052540,0.597500,1.302498,2.899998,4.919998,7.049999,1748362400,AAPL
2019-03-18,47.005001,104879200,46.450001,47.097500,46.447498,41.72940,42.569162,78.724282,41.72940,47.174667,...,0.721072,98.041317,96.240216,0.475002,1.577499,3.880001,5.375000,11.457500,1853241600,AAPL
2019-03-19,46.632500,126585600,47.087502,47.247501,46.480000,41.92075,42.728509,73.527018,41.92075,47.369412,...,0.735358,87.378112,92.877692,-0.372501,0.700001,3.404999,3.820000,9.567501,1726656000,AAPL
2019-03-20,47.040001,124140800,46.557499,47.372501,46.182499,42.12190,42.897587,80.396901,42.12190,47.569044,...,0.784822,93.346666,92.922032,0.407501,0.510002,2.315002,3.495003,10.057503,1850796800,AAPL


In [3]:
# Now let's just select our subject stock information in the APPL stock.
nvda_data = core_stock_data[core_stock_data['Ticker'] == 'NVDA']
print(nvda_data.head())

            Close_core  Volume_core  Open_core  High_core  Low_core  SMA_core  \
Date                                                                            
2019-03-14     4.13900    519384000    4.22475    4.23675   4.13125  3.767420   
2019-03-15     4.24525    661484000    4.18250    4.28825   4.17900  3.784215   
2019-03-18     4.22375    486992000    4.27700    4.33125   4.19650  3.804695   
2019-03-19     4.39275    854972000    4.31900    4.43750   4.30200  3.824455   
2019-03-20     4.36000    719148000    4.42200    4.47575   4.32500  3.839955   

            EMA_core   RSI_core  BBM_core  BBU_core  ...  ATR_14_core  \
Date                                                 ...                
2019-03-14  3.787336  58.518319  3.767420  4.191424  ...     0.136232   
2019-03-15  3.805293  63.514823  3.784215  4.216157  ...     0.134500   
2019-03-18  3.821704  64.662206  3.804695  4.220324  ...     0.135179   
2019-03-19  3.844097  72.317478  3.824455  4.256109  ...     0.1424

#### Note above that the starting date is 03-14, this is because of the rolling windows when we calculated our SMA (Single Moving Average).  With our setting of 50 days for the window it was offset by the starting date of 01-01-2019, and we needed a whole window to compute.  So the date you see above in 03-14 starts a new window for us.

#### Let's begin our EDA analysis with a simple look at Closing price over time.  We will be using Plotly a lot here, as we can use the interactivity of a singular plot to make multiple insights.

In [4]:
x = np.arange(len(nvda_data))
y = nvda_data['Close_core'].values

slope, intercept, r_value, p_value, std_err = linregress(x, y)

regression_line = slope * x + intercept


fig = go.Figure()

fig.add_trace(go.Scatter(x = nvda_data.index, y = nvda_data['Close_core'], mode = 'lines', name = 'Close Price'))

fig.add_trace(go.Scatter(x = nvda_data.index, y = regression_line, mode = 'lines', name = 'Linear Trend', line = dict(color = 'red', dash = 'dash')))

fig.update_layout(title = 'NVDA Closing Price with Linear Trend Line', xaxis_title = 'Date', yaxis_title = 'Price', template = 'plotly_dark')

fig.show()

#### Key Takeaways:  NVDA experienced breakout growth in pricing during our observation period.  Looking into historical events we can see:
#### (2019 - 2020) Between 2019 and 2020 AAPL had slowing iPhone sales which led to a direct impact on its stock performance.  Also with the trade tensions between US and China (which AAPL relies heavily on) the stock took a hit as a result.

#### (2020 - 2023) Coming out of the pandemic AAPL began to recover rapidly as everyone began to go back to work.  There was a surge of demand for their products and it was reflected in their corresponding stock prices.  Between these events, their 2020 Q3 stock split, and the anticipation surrounding the 5G iPhone launch AAPL had an excellent period of strong performance.

#### (2023 - end of data (06-30-2024)) This period is a bit of a mixed bag for AAPL, and with reasons.  On the positive there is the continuing AI Surge that has contributed positively, as well as their continued new product launches.  However there are the looming fears of a recession as well as continued inflation and high interest rates (amongst other things) that have kept AAPL from doing more in this period.

#### Now let's look at Volume over the same period for NVDA.

In [5]:
# converting to monthly data for a smoother plot
nvda_monthly = nvda_data.resample('ME').sum()

fig = go.Figure()

fig.add_trace(go.Bar(x = nvda_monthly.index, y = nvda_monthly['Volume_core'], name = 'Monthly Volume', marker_color = 'cyan'))

fig.update_layout(title = 'NVDA Monthly Volume Over Time', xaxis_title = 'Date', yaxis_title = 'Volume', template = 'plotly_dark')

fig.show()

#### Looking at the above monthly Volume for NVDA we can see a couple things.  The time period for 2020 - 2021 again is noticeable, only this time its because NVDA its just an avg trading year for them.  While we do see their peak trading months at this time in our viewing window, NVDA differs in our core stocks in that they mostly maintained this level of volume aside from 2021.

#### Now let's look at the SMA and EMA (Simple Moving Average and Exponential Moving Average, respectively) for NVDA, to see what trends exist in this time period.

In [6]:
fig = go.Figure()

# We will again use Close Price here as a starting figure for our SMA and EMA
fig.add_trace(go.Scatter(x = nvda_data.index, y = nvda_data['Close_core'], mode = 'lines', name = 'Close Price'))

# Plot our SMA
if 'SMA_core' in nvda_data.columns:
    fig.add_trace(go.Scatter(x = nvda_data.index, y = nvda_data['SMA_core'], mode = 'lines', name = 'SMA 50'))

# Plot our EMA
if 'EMA_core' in nvda_data.columns:
    fig.add_trace(go.Scatter(x = nvda_data.index, y = nvda_data['EMA_core'], mode = 'lines', name = 'EMA 50'))

fig.update_layout(title = 'NVDA Closing Price with SMA and EMA', xaxis_title = 'Date', yaxis_title = 'Price', template = 'plotly_dark')
fig.show()

#### Let's take a look at this one, as it introduces some new concepts.  Only looking at the SMA and EMA lines for a second (the red and the green lines) if both of them are sloping upwards it can indicate a short-term uptrend in the price and and confirm a bullish momentum.  We can clearly see this in several positions on our plot, notably in almost the entirety of 2023 and 2024.  There are smaller examples but these two are the most obvious.  Conversely if these lines are together moving down then it can indicate a short-term downtrend or bearish momentum.

#### Now if the Closing Price is above our SMA and EMA lines then it indicates that the stock is trending above the recent average and shows suggested continued strength.  This can also be illustrated in the same periods noted above.

#### Looking at SMA vs EMA, if the SMA is *above* the EMA line this suggests that the the recent pricing is weaker and can indicate a price slow or momentum downshift.  Conversely if the EMA line is above the SMA line it can indicate a positive price shift and momentum upturn, which we can see at times such as Nov 5 - Dec 17 of 2022 and May 30 - Jul 18 of 2024.

#### Now let's look at our RSI (Relative Strength Index) for our NVDA data.

In [8]:
# Converting to monthly data for a smoother plot
nvda_monthly = nvda_data.resample('ME').agg({
    'Close_core' : 'last',
    'RSI_core' : 'last'
})

fig = go.Figure()

# Plotting our RSI line
fig.add_trace(go.Scatter(x = nvda_monthly.index, y = nvda_monthly['RSI_core'], mode = 'lines', name = 'RSI'))

# Now adding lines for Overbought and Oversold at 0.7 and 0.3 respectively
fig.add_trace(go.Scatter(x = nvda_monthly.index, y = [70]*len(nvda_monthly), mode = 'lines', name = 'Overbought (70)', line = dict(dash = 'dash', color = 'red')))
fig.add_trace(go.Scatter(x = nvda_monthly.index, y = [30]*len(nvda_monthly), mode = 'lines', name = 'Oversold (30)', line = dict(dash = 'dash', color = 'green')))

fig.update_layout(title = 'NVDA Monthly RSI Over Time', xaxis_title = 'Date', yaxis_title = 'RSI', template = 'plotly_dark')

fig.show()

#### This plot above is very interesting as it can show us potential price shifts.  The red line for Overbought indicates that while the price has been strong it could be due for a reversal or a decrease.  You can see this happen multiple times over this plot with the sudden spike over the red line, then a quick dip back below.  This can take a while to happen, but eventually does.

#### Let's also look at the Oversold line.  Similarly to the Overbought line this line indicates when a positive shift in price is about to happen.  So when the RSI value dips below the green line the signs point toward an increase in pricing.  This can also be viewed in the plot numerous times, especially in Apr - Jun of 2019 and early Aug - Sept of 2022.

#### Now let's make use of some of the other features in our dataset and make a Candlestick Chart.

In [9]:
# We will again be using monthly sampling for interpretability
nvda_monthly_candles = nvda_data.resample('ME').agg({
    'Open_core' : 'first',
    'High_core' : 'max',
    'Low_core' : 'min',
    'Close_core' : 'last',
    'Volume_core' : 'sum',
    'SMA_core' : 'last',
    'EMA_core' : 'last'
})

# Let's start by compiling the features we need for this one.
fig = go.Figure(data = [go.Candlestick(x = nvda_monthly_candles.index,
                open = nvda_monthly_candles['Open_core'],
                high = nvda_monthly_candles['High_core'],
                low = nvda_monthly_candles['Low_core'],
                close = nvda_monthly_candles['Close_core'],
                name = 'Candlesticks')])

# Now adding in the SMA again
fig.add_trace(go.Scatter(x = nvda_monthly_candles.index, y = nvda_monthly_candles['SMA_core'], mode = 'lines', name = 'SMA 50'))

# Adding in the EMA as well
fig.add_trace(go.Scatter(x = nvda_monthly_candles.index, y = nvda_monthly_candles['EMA_core'], mode = 'lines', name = 'EMA 50'))

fig.update_layout(title = 'NVDA Candlestick Chart with SMA and EMA', xaxis_title = 'Date', yaxis_title = 'Price', template = 'plotly_dark')

fig.show()

#### Candlestick charts are great at showing a lot of information.  The size of the candle shows the range of pricing in the given window, in our case a month.  The color (green for positive change, red for negative) will dictate how the final closing price was settled (final closing - beginning opening price for the total window, again a month here).  

#### You can then begin to see trends just by noticing the colors, although there are other parts of the candlestick too.  You can notice buyer/seller behavior by looking at successive green or red candlesticks.  If you see multiple long red candlesticks together it could mean that sellers are pushing prices lower.  This can be demonstrated in NVDA from Dec 2021 - Feb 2022.  Conversely successive green candlesticks can show buyer behavior pushing positive price changes.  This can be shown in several places, especially from Jan - Aug of 2023.

#### Now let's look at a correlation heatmap to see which indicators are most closely related to price movements for our NVDA stock.

In [10]:
corr_matrix = nvda_data[['Open_core', 'High_core', 'Low_core', 'Volume_core','Close_core', 'EMA_core', 'SMA_core', 'RSI_core', 'BBM_core', 'BBL_core', 'BBU_core', 'MACD_core', 'MACD_Signal_core', 'MACD_Hist_core', 'ADX_14_core', 'CCI_20_core', 'ATR_14_core', 'Stoch_K_core', 'Stoch_D_core', 'Momentum_1_core', 'Momentum_3_core', 'Momentum_7_core', 'Momentum_30_core', 'Momentum_50_core', 'OBV_core']].corr()

fig = px.imshow(corr_matrix, text_auto = True, aspect = 'auto', color_continuous_scale= 'Viridis')

fig.update_layout(title = 'Correlation Matrix of NVDA Features', template = 'plotly_dark')
fig.show()

#### For the above correlation plot we are looking at which features correlate the strongest with our Close price, as that is going to be our strongest driver for this project.  Looking at our colorbar a score of 1 is very strong, and this chart shows that our SMA_core and EMA_core have very strong correlations with our Close Price as well as all of the Bollinger Band features.  While there are multiple poorly correlated features with our Close Price they can still provide value and help discover trends when we get to our modeling.

#### Now let's look at a distribution of daily returns for NVDA.

In [12]:
# Calculating the pct_change of the Close column
nvda_data = nvda_data.copy()
nvda_data.loc[:, 'daily_return'] = nvda_data['Close_core'].pct_change()

# Plotting this new feature, with n = 50 bins as a default
daily_returns = nvda_data['daily_return'].dropna()

# Calculating the KDE for the daily returns
kde = gaussian_kde(daily_returns)
x_vals = np.linspace(daily_returns.min(), daily_returns.max(), 1000)
kde_vals = kde(x_vals)

# Calculating the histogram first without plotting
hist_values, bin_edges = np.histogram(daily_returns, bins = 50)

# Calculate the bin width
manual_scaling_factor = max(hist_values) / max(kde_vals) * 1.3

# Plotting histogram of daily returns
fig = go.Figure(data = [go.Histogram(x = daily_returns, nbinsx=50, name = 'Histogram', marker_color = 'blue', opacity = 0.6)])

# Scaling for the KDE Curve is needed here
#scaling_factor = max(hist_values) / max(kde_vals)
kde_vals_scaled = kde_vals * manual_scaling_factor

# Plotting the KDE Curve
fig.add_trace(go.Scatter(x = x_vals, y = kde_vals_scaled, mode = 'lines', name = 'KDE', line = dict(color = 'red')))

fig.update_layout(title = 'Distribution of NVDA Daily Returns with KDE', xaxis_title = 'Daily Return', yaxis_title = 'Frequency', template = 'plotly_dark')
fig.show()

#### There are some key takeaways from our distribution chart.  The first one is that the distribution is mostly centered around 0, with very little spread from the center.  With this near symmetry it suggests that positive and negative are both likely and that there is no strong bias in the direction of the returns.  Also with it being a normal distribution it shows that our NVDA stock behaves in a predictable manner, where extreme returns are rare.  The minimal spread (distance away from 0) indicates low volatility as well.

#### Let's now look at a Rolling Mean and Volatility plot.  We will use this to understand the stability of price movements over time, as it is helpful to identify periods of high uncertainty and/or strong trends in our pricing.

In [13]:
### First let's create the rolling mean and rolling std needed for this plot.
#### We will keep the same window size as our SMA and EMA windows for consistency and to also help us as we view the long-term analysis.
nvda_data = nvda_data.copy()

nvda_data.loc[:, 'Rolling_Mean'] = nvda_data['Close_core'].rolling(window = 50).mean()
nvda_data.loc[:, 'Rolling_Std'] = nvda_data['Close_core'].rolling(window = 50).std()

fig = go.Figure()

# Plot the Rolling Mean on primary y-axis
fig.add_trace(go.Scatter(x = nvda_data.index, y = nvda_data['Rolling_Mean'], mode = 'lines', name = 'Rolling Mean'))

# Plot the Rolling Std (Volatility) on secondary y-axis
fig.add_trace(go.Scatter(x = nvda_data.index, y = nvda_data['Rolling_Std'], mode = 'lines', name = 'Rolling Std (Volatility)', line = dict(dash = 'dash'), yaxis = 'y2'))

fig.update_layout(title = 'NVDA Rolling Mean and Volatility',
                xaxis_title = 'Date',
                yaxis_title = 'Price',
                yaxis2 = dict(
                    title = 'Volatility (Rolling Std)',
                    overlaying = 'y',
                    side = 'right'
                ),
                template = 'plotly_dark'
)

fig.show()

#### In this plot above we can see again the massive price increase over time in the blue line that is the Rolling Mean.  Our previous plots have shown this as well a bit this is just more reinforcement and a bit more demonstrative of that fact.  The red line in the Rolling Std is a bit more important as it displays Volatility.  Our window size of 50 days plays a part in this too as it will smooth out short-term volatility and provide a longer-term view of that price stability.  As we can see the price for NVDA is quite stable as long as there aren't many movements, yet note the volatility increase(s) as the price does after 2023.

#### We will now look at a few of the plots whose features correlated most closely with our target Price.  We will take these last few plots and look at some Bollinger Bands and On-Balance Volume for our NVDA data.

In [14]:
obv_trace = go.Scatter(x = nvda_data.index, y = nvda_data['OBV_core'], mode = 'lines', name = 'OBV')

close_trace = go.Scatter(x = nvda_data.index, y = nvda_data['Close_core'], mode = 'lines', name = 'Close Price', yaxis = 'y2')

layout = go.Layout(
    title = "On-Balance Volume (OBV) and Close Price for AAPL",
    xaxis = dict(title = 'Date'),
    yaxis = dict(title = 'OBV'),
    yaxis2 = dict(title = 'Close Price', overlaying = 'y', side = 'right'),
    legend = dict(x = 0.1, y = 0.9),
    template = 'plotly_dark'
)

fig = go.Figure(data = [obv_trace, close_trace], layout = layout)

fig.show()

#### OBV (On-Balance Volume) can provide valuable insights into the relationship between price movements and trading volume.  We can show the confirmation of the trend when the OBV line is moving in the same direction as the Price line (for the second half of the plot mostly) as it would suggest that the price trend is backed by the volume, indicating the momentum direction.
#### OBV often acts as a leading indicator and predictor of the Price, as you can see above in large portions of the plot it mirrors our Price movements and is just ahead (predicted) just before the actual time period.  Especially between 2021 - start of 2024 they do a decent job of mirroring each other.

#### For our last plot in our EDA notebook we will look at all 3 of the Bollinger Bands with a Close price overlay.  The Bollinger Bands were our other high correlator in our correlation matrix above.  Let's see what we can derive from it.

In [15]:
trace_bbu = go.Scatter(x = nvda_data.index, y = nvda_data['BBU_core'], mode = 'lines', name = 'Upper Bollinger Band', line = dict(width = 2,color = 'green'), opacity = 0.4)
trace_bbm = go.Scatter(x = nvda_data.index, y = nvda_data['BBM_core'], mode = 'lines', name = 'Middle Bollinger Band', line = dict(width = 2, color = 'orange'), opacity = 0.6)
trace_bbl = go.Scatter(x = nvda_data.index, y = nvda_data['BBL_core'], mode = 'lines', name = 'Lower Bollinger Band', line = dict(width = 2, color = 'red'), opacity = 0.4)
trace_close = go.Scatter(x = nvda_data.index, y = nvda_data['Close_core'], mode = 'lines', name = 'Close Price', line = dict(width = 3, color = 'blue'))

data = [trace_bbu, trace_bbm, trace_bbl, trace_close]

layout = go.Layout(title = "Bollinger Bands with Close Price for NVDA",
                xaxis = dict(title = "Date"),
                yaxis = dict(title = "Price"),
                template = 'plotly_dark')

fig = go.Figure(data = data, layout = layout)

fig.show()

#### The above Bollinger Bands plot with Closing Price combines a lot of features and information we have seen in this notebook to give a clear picture of the stock data and whether it is relatively high or low.  The Close price will oscillate between the Upper and Lower bands (Green and Red, respectively).  Now when the Close Price touches these bands and moves through it, this can indicate Overbought or Oversold conditions (much like our RSI plot above).
#### The width of the bands can also show volatility, where a wider band visual indicates increased volatility and conversely shortened or decreased band width can indicated less volatility for that time period.
#### What to look out for is when the Price does indeed break outside of the bands, as that is when something unique or unusual is happening.  In the above while the Price for AAPL touches the outside bands multiple times, AAPL's price only exceeds the bands a few times: Nov 1 - 9 of 2021 and May 23 - Jun 18 of 2024.  

## Summary of Findings for NVDA
#### We have taken a good look at our NVDA stock covering the given time period.  Here is some of the key takeaways from our plots:

#### -NVDA Stock Price has shown to be an aggressive performer with volatility that reflects its performance in recent years.
#### -AAPL's Closing Price is strongly correlated with the SMA (Single Moving Average) and EMA (Exponential Moving Average) as well as Bollinger Band data.
#### -AAPL's price behaves in a somewhat predictable manner, due to the OBV.